In [51]:
""" funcion auxiliar para las multiples traslaciones """
function muvoltint( x, factor, offset )
    result = ( x / factor ) - offset;
end
using Statistics
""" buscasatura( datos::Array; 
        factor, offset = -2048, freq = deffreq, tini = 0.5, tfin = 10, umbral = 1900, 
        tol = 0.04 )
    Busca saturados por promedio sobre umbral 
"""
function buscasatura( 
        datos::Array; factor, offset = -2048, freq = freq, tini = 0.5, tfin = 10, 
        umbral = 1900, tol = 0.04 )
    satint = muvoltint( umbral, factor, offset ) # pasamos el valor a los enteros
    ( nchans, nc ) = size( datos );
    cini = round( Int, ceil( tini * freq ) );
    cfin = round( Int, ceil( tfin * freq ) );
    result = Set( Array{ Int16, 1 }[ ] );
    for j = 1:nchans
        if factor > 0
            sepasa = findall( x -> x > satint, datos[ j, cini : cfin ] );
        else     
            sepasa = findall( x-> x < satint, datos[ j, cini : cfin ] );
        end
        enemalos = length( sepasa );
        enetotal = length( cini : cfin );
        if ( enemalos / enetotal ) > tol
            ( a, b ) = divrem( j, 64 )
            a += 1;
            bla = [ j, a, b ];
            result = push!( result, bla );
        end
    end
    return result
end

using Statistics
function buscastdraras( 
        datos::Array; tini, tfin, factor, offset = -2048, freq = deffreq, bajo = 10, 
        alto = 40 )
    # ventms es la ventana en milisegundos
    # busca saturardos por desviación por ventana por umbral
    bai = abs( bajo / factor );
    ari = abs( alto / factor );
    ( nchans, nc ) = size( datos );
    cini = round( Int, ceil( freq * tini ) );
    cfin = round( Int, ceil( freq * tfin ) );
    result = Set( Array{ Int16, 1 }[ ] );
    for j = 1:nchans        
        sigma = std( datos[ j, cini : cfin ] );    
        if sigma > ari || sigma < bai
            ( a, b ) = divrem( j, 64 );
            a += 1;
            bla = [ j, a, b ];
            result = push!( result, bla );
        end
    end
    return result
end
function buscaCanalPicos( 
        datos::Array; factor, offset = -2048, tini = 0.5, tfin = 8, freq = deffreq, 
        maxvolt = -100, minvolt = -1500, minstd = 10, maxstd = 35 )
    # Busquemos los canales con probable respuesta de potencial de accion
    ( nchans, nc ) = size( datos );
    taux1 = round( Int, ceil( tini * freq ) );
    taux2 = round( Int, ceil( tfin * freq ) );
    mxvint = muvoltint( maxvolt, factor, offset );
    mnvint = muvoltint( minvolt, factor, offset );
    ustdint = abs( maxstd / factor );
    lstdint = abs( minstd / factor );
    println( "Estoy buscando del cuadro " , taux1, " al , ", taux2 )
    result = Set( Array{ Int16, 1 }[ ] );
    for j = 1:nchans
        fondo = minimum( vec( datos[ j, taux1 : taux2 ] ) );
        dpgs = std( datos[ j, taux1 : taux2 ] );
        test = true;
        if factor > 0
            test = ( mxvint > fondo > mnvint ) && ( ustdint > dpgs > lstdint );
        else
            test = ( mxvint < fondo < mnvint ) && ( ustdint > dpgs > lstdint );
        end
        if test
            ( a, b ) = divrem( j, 64 );
            #los canales se cuentas desde 1, no de cero
            a += 1;
            b += 1;       
            bla = [ j, a, b ];     
            result = push!( result, bla );
        end
    end
    #=  Esta rutina no es muy confiable. Solo da buenos resultados
    con actividad evocada. Necesitamos algo mas estricto  =#
    return result
end
function elemtorow( xxs::Set; n = 2, renglones = false )
    # pasa los elementos de un conjunto a rengoles de un array
    result = zeros( UInt16, 1, n );
    for k in xxs
        if renglones
            result = vcat( result, k );
        else
            result = vcat( result, transpose( k ) )
        end
    end
    return result[ 2 : end, : ]
end

elemtorow (generic function with 1 method)

In [3]:
using JLD

In [5]:
FILEBRW = "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10.brw"
PATHMAIN = split( FILEBRW, "." )[ 1 ];
FILEPATHS = joinpath( PATHMAIN, "Info", "Paths.jld" );
PATHS = load( FILEPATHS )[ "PATHS" ];
PATHINFO = PATHS[ "PATHINFO" ];
FILEVARS = joinpath( PATHINFO, "Variables.jld" );
Variables = load( FILEVARS )[ "Variables" ];

In [10]:
Segment = load( 
    "/run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10/Voltage/BIN64.jld" 
    );

In [14]:
Factor = Variables[ "SignalInversion" ] * ( 
    Variables[ "MaxVolt" ] - Variables[ "MinVolt" ] ) / ( 2^Variables[ "BitDepth" ] );

In [18]:
NRecFrames = size( Segment[ "data" ], 2 );

In [44]:
arxname = FILEBRW;
arx = Variables;
factor = Factor;
dataraw = Float64.( Segment[ "data" ] );

In [8]:
freq = arx[ "SamplingRate" ] / 1000; # usamos kHz y milisegundos aqui porfa
retrazo, final, latencia = 0, 0, 0;

In [30]:
finbusqueda = NRecFrames/freq;

In [45]:
#=  los parametros son datos promediados, frecuencia, tiempo post estimulo en ms, 
umbral en microvolts, umbral de saturación en microvolts  =#
iniciobusqueda = 0.01;
# finbusqueda = 282;
# finbusqueda = NRecFrames;
PruebaRespuesta = buscaCanalPicos( 
    dataraw, factor = factor, tini = iniciobusqueda, tfin = finbusqueda, maxvolt = -300, 
    freq = freq );

Estoy buscando del cuadro 1 al , 49446


In [46]:
# y luego los saturados
satu = buscasatura( 
    dataraw, factor = factor, freq = freq, tini = iniciobusqueda, tfin = finbusqueda, 
    umbral = 2000 );

In [47]:
numerosaturados = length( satu );
println( "Encontramos ", numerosaturados, " canales probablemente saturados." );
println( "Encontramos ", length( PruebaRespuesta ), " canales probablemente con actividad." 
);

Encontramos 11 canales probablemente saturados.
Encontramos 0 canales probablemente con actividad.


In [48]:
quietos = buscastdraras(
    dataraw, tini = iniciobusqueda, tfin = finbusqueda, factor = factor, freq = freq, 
    bajo = 16, alto = 10000 );

In [49]:
intensos = buscastdraras(
    dataraw, tini = iniciobusqueda, tfin = finbusqueda, factor = factor, freq = freq, 
    bajo = 0, alto = 30 );
numerosaturados = length( quietos );
println( "Encontramos ", numerosaturados, " canales muy quietos" );
numerosaturados = length( intensos );
println( "Encontramos ", numerosaturados, " canales muy intensos" );

Encontramos 1 canales muy quietos
Encontramos 4078 canales muy intensos


In [52]:
mal = union( quietos, satu );
buenos = union( intensos, PruebaRespuesta );
malforsave = elemtorow( mal, n = 3 );
buenforsave = elemtorow( buenos, n = 3 );

In [59]:
dirgen = dirname( arxname ) * "/";
basegen = basename( arxname )[ 1 : ( end - 4 ) ];
# una palabra para indicar el intervalo
outname = string( dirgen, basegen, "_BIN64_preproc.h5" );

false

In [78]:
using HDF5
# aqui hay que arreglar el rollo del directorio
listaextra = Dict(
    "CanalesBuenos" => buenforsave,
    "CanalesMalos"  => malforsave,
    "Datos"         => dataraw
);
h5open( outname, "w" ) do file
    for nam in keys( arx )
        datos = arx[ nam ];
        println( "abriendo los datos ", nam );
        if isa( datos, Real ) || isa( datos, String ) 
            write( file, "$nam", datos );
        end
    end
    for k in keys( listaextra )
        datos = listaextra[ k ];
        println( "abriendo los datos ", k );
        write( file, "$k", datos );
    end
    
end
println( "hemos guardado este cacho en el archivo ", outname );

abriendo los datos NRecFrames
abriendo los datos BRWname
abriendo los datos BRWsizeGB
abriendo los datos STD
abriendo los datos SamplingRate
abriendo los datos Version
abriendo los datos StdMean
abriendo los datos nChs
abriendo los datos FwVersion
abriendo los datos Raw
abriendo los datos MinVolt
abriendo los datos MeaType
abriendo los datos MaxVolt
abriendo los datos Chs
abriendo los datos BitDepth
abriendo los datos MeaLayersInfo
abriendo los datos ExperimentType
abriendo los datos NRows
abriendo los datos System
abriendo los datos Description
abriendo los datos SignalInversion
abriendo los datos GUID
abriendo los datos HwVersion
abriendo los datos SysChs
abriendo los datos ValidChs
abriendo los datos NCols
abriendo los datos Layout
abriendo los datos CanalesMalos
abriendo los datos Datos
abriendo los datos CanalesBuenos
hemos guardado este cacho en el archivo /run/media/isabel/Data/Hipocampo/Espontanea/01-06-2022/BRWs/Phase_10_BIN64_preproc.h5


### las variables de los canales buenos y malos estan en UInt...